In [6]:
import csv
import json
import os
import numpy as np
import nibabel as nib
import pandas as pd
import suspect

# Make a DataFrame with .rda headers' data

In [7]:
from suspect import MRSData, transformation_matrix

import numpy
import struct
import re

# The RDA format consists of a large number of key value pairs followed by raw
# data. The values need to be cast into different datatypes depending on the
# key, this dictionary stores a mapping of key to datatype.

rda_types = {
    "floats": ["PatientWeight", "TR", "TE", "TM", "TI", "DwellTime", "NumberOfAverages",
               "MRFrequency", "MagneticFieldStrength", "FlipAngle", "SliceThickness",
               "FoVHeight", "FoVWidth", "PercentOfRectFoV", "PixelSpacingRow",
               "PixelSpacingCol", "VOIPositionSag", "VOIPositionCor",
               "VOIPositionTra", "VOIThickness", "VOIPhaseFOV", "VOIReadoutFOV",
               "VOIReadoutVOV", "VOINormalSag", "VOINormalCor", "VOINormalTra",
               "VOIRotationInPlane", "FoV3D", "PixelSpacing3D"],
    "integers": ["SeriesNumber", "InstanceNumber", "AcquisitionNumber", "NumOfPhaseEncodingSteps",
                 "NumberOfRows", "NumberOfColumns", "VectorSize", "EchoNumber",
                 "NumberOf3DParts", "HammingFilterWidth", "NumberOfEchoes"],
    "strings": ["PatientID", "PatientName", "StudyDescription", "PatientBirthDate",
                "StudyDate", "StudyTime", "PatientAge", "SeriesDate", "SeriesTime",
                "SeriesDescription", "ProtocolName", "PatientPosition", "ModelName",
                "StationName", "InstitutionName", "DeviceSerialNumber", "InstanceDate",
                "InstanceTime", "InstanceComments", "SequenceName", "SequenceDescription",
                "Nucleus", "TransmitCoil", "PatientSex", "HammingFilter", "FrequencyCorrection"],
    "float_arrays": ["PositionVector", "RowVector", "ColumnVector"],
    "integer_arrays": ["CSIMatrixSize", "CSIMatrixSizeOfScan", "CSIGridShift"],
    "string_arrays": ["SoftwareVersion"],
    "dictionaries": ["TransmitRefAmplitude"]
}


def load_rda(json_el):
    global rda_types
    header_dict = {}
    for i in json_el:
        key = i
        value = json_el[i]
        if key in rda_types["strings"]:
            header_dict[key] = value
        elif key in rda_types["integers"]:
            header_dict[key] = int(value)
        elif key in rda_types["floats"]:
            header_dict[key] = float(value)
        elif "[" in key and "]" in key:
            # could be a dict or a list
            key, index = re.split("\]|\[", key)[0:2]
            if key in rda_types["dictionaries"]:
                if key not in header_dict:
                    header_dict[key] = {}
                header_dict[key][index] = value
            else:
                # not a dictionary, must be a list
                if key in rda_types["float_arrays"]:
                    value = float(value)
                elif key in rda_types["integer_arrays"]:
                    value = int(value)
                index = int(index)
                # make sure there is a list in the header_dict, with enough entries
                if not key in header_dict:
                    header_dict[key] = []
                while len(header_dict[key]) <= index:
                    header_dict[key].append(0)
                header_dict[key][index] = value
        # now we can read the data
        complex_data = []


    # some .rda files have a misnamed field, correct this here
    if "VOIReadoutFOV" not in header_dict:
        if "VOIReadoutVOV" in header_dict:
            header_dict["VOIReadoutFOV"] = header_dict.pop("VOIReadoutVOV")

    # combine positional elements in the header
    voi_size = (header_dict["VOIReadoutFOV"],
                header_dict["VOIPhaseFOV"],
                header_dict["VOIThickness"])
    voi_center = (header_dict["VOIPositionSag"],
                  header_dict["VOIPositionCor"],
                  header_dict["VOIPositionTra"])
    voxel_size = (header_dict["PixelSpacingCol"],
                  header_dict["PixelSpacingRow"],
                  header_dict["PixelSpacing3D"])

    x_vector = numpy.array(header_dict["RowVector"])
    y_vector = numpy.array(header_dict["ColumnVector"])

    to_scanner = transformation_matrix(x_vector, y_vector, numpy.array(voi_center), voxel_size)

    # put useful components from the header in the metadata
    metadata = {
        "voi_size": voi_size,
        "position": voi_center,
        "voxel_size": voxel_size,
        "protocol": header_dict["ProtocolName"],
        "to_scanner": to_scanner,
        "from_scanner": numpy.linalg.inv(to_scanner)
    }

    return MRSData(complex_data,
                   header_dict["DwellTime"] * 1e-6,
                   header_dict["MRFrequency"],
                   te=header_dict["TE"],
                   transform=to_scanner,
                   metadata=metadata)


In [8]:
 csv_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/rda_headers.csv'

In [14]:
import pandas as pd

df = pd.read_csv(csv_path, sep=',', header=0).drop(columns='Unnamed: 0')
jsn = json.loads(df.to_json(orient="records"))

In [15]:
%%bash
#jupyter nbextension enable --py --sys-prefix qgridimport qgrid# only required if you have not enabled the ipywidgets nbextension yet
#jupyter nbextension enable --py --sys-prefix widgetsnbextension#to show a df simply use the below:

In [16]:
import qgrid
qgrid.show_grid(df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [20]:
df[df.PatientID=='NeuroMet033'][['PatientID', 'VOIPositionSag', 'VOIPositionCor', 'VOIPositionTra']]

,PatientID,VOIPositionSag,VOIPositionCor,VOIPositionTra
100,NeuroMet033,28.7,-16.8,-10.0
101,NeuroMet033,28.7,-16.8,-10.0
102,NeuroMet033,28.7,-16.8,-10.0


In [21]:
df2 = df.iloc[[148, 14, 210, 187, 121, 142, 109, 115, 289, 342]]

In [22]:
df2

,PatientName,PatientID,PatientSex,PatientBirthDate,StudyDate,StudyTime,StudyDescription,PatientAge,PatientWeight,SeriesDate,...,FoVHeight,FoVWidth,FoV3D,PercentOfRectFoV,NumberOfRows,NumberOfColumns,NumberOf3DParts,PixelSpacingRow,PixelSpacingCol,PixelSpacing3D
148,MDC-0131_NeuroMet045,NeuroMet045,M,19431003,20170628,114505.953,PTB^Ariane,073Y,70.0,20170628,...,20.0,20.0,20.0,1.0,1,1,1,20.0,20.0,20.0
14,MDC-0131_NeuroMet004,_NeuroMet004,F,19440927,20161123,110853.500,PTB^Ariane,072Y,68.0,20161123,...,20.0,20.0,12.0,1.0,1,1,1,20.0,20.0,12.0
210,MDC-0131_NeuroMet061,NeuroMet061,M,19450626,20171122,104155.812,PTB^Ariane,072Y,79.0,20171122,...,20.0,20.0,20.0,1.0,1,1,1,20.0,20.0,20.0
187,MDC-0131_NeuroMet048,NeuroMet048,F,19491231,20170913,152418.156,PTB^Ariane,067Y,65.0,20170913,...,20.0,20.0,20.0,1.0,1,1,1,20.0,20.0,20.0
121,MDC-0131_NeuroMet039,NeuroMet039,F,19541005,20170614,114901.156,PTB^Ariane,062Y,88.0,20170614,...,20.0,20.0,20.0,1.0,1,1,1,20.0,20.0,20.0
142,MDC-0131_NeuroMet037,NeuroMet037,F,19600717,20170628,94254.984,PTB^Ariane,056Y,71.0,20170628,...,20.0,20.0,20.0,1.0,1,1,1,20.0,20.0,20.0
109,MDC-0131_NeuroMet036,NeuroMet036,F,19540205,20170531,101151.734,PTB^Ariane,063Y,77.0,20170531,...,20.0,20.0,20.0,1.0,1,1,1,20.0,20.0,20.0
115,MDC-0131_NeuroMet034,NeuroMet034,F,19430921,20170614,102112.953,PTB^Ariane,073Y,85.0,20170614,...,20.0,20.0,20.0,1.0,1,1,1,20.0,20.0,20.0
289,MDC-0131_NeuroMET080,NeuroMET080,M,19470118,20180606,112536.250,PTB^Ariane,071Y,86.0,20180606,...,20.0,20.0,20.0,1.0,1,1,1,20.0,20.0,20.0
342,MDC-0131_NeuroMET090,NeuroMET090,M,19430615,20190626,105543.250,PTB^Ariane,076Y,71.0,20190626,...,20.0,20.0,20.0,1.0,1,1,1,20.0,20.0,20.0


In [23]:
df=df2

# 2 Make a Nifty with the MRS Voxel as 10mm radius sphere

In [24]:
# Credits NLTools: https://github.com/cosanlab/nltools
def sphere(nii_path, r, p):
    """ create a sphere of given radius at some point p in the brain mask
    Args:
        r: radius of the sphere
        p: point (in coordinates of the brain mask) of the center of the sphere
    """
    nii = nib.load(nii_path)
    dims = nii.shape

    x, y, z = np.ogrid[-p[0]:dims[0]-p[0], -p[1]:dims[1]-p[1], -p[2]:dims[2]-p[2]]
    mask = x*x + y*y + z*z <= r*r

    activation = np.zeros(dims)
    #print(activation.sum())
    activation[mask] = 1
    #print(activation.sum())
    #activation = np.multiply(activation, nii.get_fdata())
    activation = activation + nii.get_fdata()
    #print(activation.sum())
    activation = nib.Nifti1Image(activation, nii.affine, nii.header)

    # return the 3D numpy matrix of zeros containing the sphere as a region of ones
    return activation

In [25]:
sphere_radius = 10

In [26]:
t1_file_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/Structural_analysis/{sub_id}/{sub_id}.mUNIbrain_DENskull_SPMmasked.nii.gz'

In [27]:
subject_space_sphere_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/Subjekt_space_spheres'
subject_space_voxel_path = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/Subject_space_Voxels'

In [ ]:
for i in jsn:
    try:
        sub_id = i['PatientID']
        t1_path = t1_file_mask.format(sub_id=sub_id)
        rda = load_rda(i)
        base_name = t1_path.split('/')[-1][:-7] 
        # get voxel coordinates for the center of mrs voxel
        t1 = suspect.image.load_nifti(t1_path)
        mrs_centre = rda.to_scanner(0, 0, 0)
        mrs_centre_index = t1.from_scanner(*mrs_centre).round().astype(int)
        # zero all voxels and put 1 for mrs voxel center
        img = nib.load(t1_path)
        new_data = img.get_fdata().copy()
        new_data[:,:,:] = 0
        new_data[mrs_centre_index[0],mrs_centre_index[1],mrs_centre_index[2]] = 1
        #create and save nifti
        new_img = nib.Nifti1Image(new_data, img.affine, img.header)
        nib.save(new_img, os.path.join(subject_space_voxel_path, base_name + '_mrs_voxel_center.nii.gz'))
        #make a sphere and save it
        sssphere = sphere(os.path.join(subject_space_voxel_path, base_name + '_mrs_voxel_center.nii.gz'),
                     r=sphere_radius,
                     p=mrs_centre_index)
        nib.save(sssphere, os.path.join(subject_space_sphere_path, base_name + '_mrs_voxel_sphere.nii.gz'))
    except:
        pass

In [ ]:
# Coregister in MNI space

In [ ]:
import ants
import os

In [ ]:
mni_std = '/usr/share/fsl/data/standard/MNI152_T1_0.5mm.nii.gz'
ants_mni_std = ants.image_read(mni_std)

In [ ]:
mov_mask = str(os.path.join(subject_space_sphere_path, '{sub_id}.mUNIbrain_DENskull_SPMmasked_mrs_voxel_sphere.nii.gz'))

In [ ]:
transform1_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MNI_coreg/{sub_id}/{sub_id}.mUNIbrain_DENskull_SPMmasked_MNIcoreg0GenericAffine.mat'
transform2_mask = '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MNI_coreg/{sub_id}/{sub_id}.mUNIbrain_DENskull_SPMmasked_MNIcoreg1Warp.nii.gz'

In [ ]:
ids = set([i['PatientID'] for i in jsn])

In [ ]:
for sub_id in ids:
    try:
        print(sub_id)
        transform1 = transform1_mask.format(sub_id=sub_id)
        transform2 = transform2_mask.format(sub_id=sub_id)
        moving = mov_mask.format(sub_id=sub_id)
        ants_mov = ants.image_read(moving)
        res = ants.apply_transforms(fixed=ants_mni_std,
                         moving=ants_mov,
                         transformlist = [transform2, transform1])
        ants.image_write(res, '/media/drive_s/AG/AG-Floeel-Imaging/02-User/NEUROMET/MRS_Voxel_masks/MNI_space_spheres/{}_MRS_Voxel_MNIwarped.nii.gz'.format(sub_id))
    except:
        pass